# BIGDATA Projet




## Importation des Donne depuis mon LakeHouses

In [2]:

from pyspark.sql import SparkSession

# Initialiser une session Spark
spark = SparkSession.builder.appName("ImportCSV").getOrCreate()

# Charger les fichiers CSV dans des DataFrames
df_customers = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("Files/olist_customers_dataset.csv")

df_sellers = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("Files/olist_sellers_dataset.csv")

df_category_translation = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("Files/product_category_name_translation.csv")

df_geolocation = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("Files/olist_geolocation_dataset.csv")

df_order_items = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("Files/olist_order_items_dataset.csv")

df_order_payments = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("Files/olist_order_payments_dataset.csv")

df_order_reviews = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("Files/olist_order_reviews_dataset.csv")

df_orders = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("Files/olist_orders_dataset.csv")

df_products = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("Files/olist_products_dataset.csv")

# Afficher les premières lignes de chaque DataFrame
df_customers.show(2)
df_sellers.show(2)
df_category_translation.show(2)
df_geolocation.show(2)
df_order_items.show(2)
df_order_payments.show(2)
df_order_reviews.show(2)
df_orders.show(2)
df_products.show(2)



StatementMeta(, e4d96399-a3a2-4659-953f-1afcb988c31b, 4, Finished, Available, Finished)

+--------------------+--------------------+------------------------+--------------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|
+--------------------+--------------------+------------------------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|                    9790|sao bernardo do c...|            SP|
+--------------------+--------------------+------------------------+--------------------+--------------+
only showing top 2 rows

+--------------------+----------------------+-----------+------------+
|           seller_id|seller_zip_code_prefix|seller_city|seller_state|
+--------------------+----------------------+-----------+------------+
|3442f8959a84dea7e...|                 13023|   campinas|          SP|
|d1b65fc7debc3361e...|                 13844| mogi guacu|    

# Nettoyage des tables 


In [3]:
# Supprimer les lignes contenant des valeurs nulles
df_orders_cleaned = df_orders.dropna(subset=["customer_id", "order_id"])


StatementMeta(, e4d96399-a3a2-4659-953f-1afcb988c31b, 5, Finished, Available, Finished)

In [4]:
# Renommer les clés primaires pour chaque table afin d'éviter toute ambiguïté

# Table des clients
df_customers_renamed = df_customers.withColumnRenamed("customer_id", "id_client") \
                                    .withColumnRenamed("customer_unique_id", "id_unique_client") \
                                    .withColumnRenamed("customer_zip_code_prefix", "code_postal_client") \
                                    .withColumnRenamed("customer_city", "ville_client") \
                                    .withColumnRenamed("customer_state", "etat_client")

# Table des vendeurs
df_sellers_renamed = df_sellers.withColumnRenamed("seller_id", "id_vendeur") \
                                .withColumnRenamed("seller_zip_code_prefix", "code_postal_vendeur") \
                                .withColumnRenamed("seller_city", "ville_vendeur") \
                                .withColumnRenamed("seller_state", "etat_vendeur")

# Table de traduction des catégories
df_category_translation_renamed = df_category_translation.withColumnRenamed("product_category_name", "categorie_produit") \
                                                         .withColumnRenamed("product_category_name_english", "categorie_produit_anglais")

# Table de géolocalisation
df_geolocation_renamed = df_geolocation.withColumnRenamed("geolocation_zip_code_prefix", "code_postal_geolocalisation") \
                                       .withColumnRenamed("geolocation_lat", "latitude_geolocalisation") \
                                       .withColumnRenamed("geolocation_lng", "longitude_geolocalisation") \
                                       .withColumnRenamed("geolocation_city", "ville_geolocalisation") \
                                       .withColumnRenamed("geolocation_state", "etat_geolocalisation")

# Table des articles de commande
df_order_items_renamed = df_order_items.withColumnRenamed("order_id", "id_commande_order_items") \
                                       .withColumnRenamed("order_item_id", "id_article_commande") \
                                       .withColumnRenamed("product_id", "id_produit") \
                                       .withColumnRenamed("seller_id", "id_vendeur") \
                                       .withColumnRenamed("shipping_limit_date", "date_limite_expedition") \
                                       .withColumnRenamed("price", "prix") \
                                       .withColumnRenamed("freight_value", "valeur_fret")

# Table des paiements de commande
df_order_payments_renamed = df_order_payments.withColumnRenamed("order_id", "id_commande_order_payments") \
                                              .withColumnRenamed("payment_sequential", "sequence_paiement") \
                                              .withColumnRenamed("payment_type", "type_paiement") \
                                              .withColumnRenamed("payment_installments", "nombre_paiements") \
                                              .withColumnRenamed("payment_value", "valeur_paiement")

# Table des avis sur les commandes
df_order_reviews_renamed = df_order_reviews.withColumnRenamed("review_id", "id_avis") \
                                            .withColumnRenamed("order_id", "id_commande_order_reviews") \
                                            .withColumnRenamed("review_score", "score_avis") \
                                            .withColumnRenamed("review_comment_title", "titre_avis") \
                                            .withColumnRenamed("review_comment_message", "message_avis") \
                                            .withColumnRenamed("review_creation_date", "date_creation_avis") \
                                            .withColumnRenamed("review_answer_timestamp", "date_reponse_avis")

# Table des commandes
df_orders_renamed = df_orders.withColumnRenamed("order_id", "id_commande_orders") \
                             .withColumnRenamed("customer_id", "id_client_orders") \
                             .withColumnRenamed("order_status", "statut_commande") \
                             .withColumnRenamed("order_purchase_timestamp", "date_achat_commande") \
                             .withColumnRenamed("order_approved_at", "date_approbation_commande") \
                             .withColumnRenamed("order_delivered_carrier_date", "date_livraison_transporteur") \
                             .withColumnRenamed("order_delivered_customer_date", "date_livraison_client") \
                             .withColumnRenamed("order_estimated_delivery_date", "date_livraison_estimee")

# Table des produits
df_products_renamed = df_products.withColumnRenamed("product_id", "id_produit") \
                                 .withColumnRenamed("product_category_name", "categorie_produit") \
                                 .withColumnRenamed("product_name_lenght", "longueur_nom_produit") \
                                 .withColumnRenamed("product_description_lenght", "longueur_description_produit") \
                                 .withColumnRenamed("product_photos_qty", "quantite_photos_produit") \
                                 .withColumnRenamed("product_weight_g", "poids_produit_grammes") \
                                 .withColumnRenamed("product_length_cm", "longueur_produit_cm") \
                                 .withColumnRenamed("product_height_cm", "hauteur_produit_cm") \
                                 .withColumnRenamed("product_width_cm", "largeur_produit_cm")


StatementMeta(, e4d96399-a3a2-4659-953f-1afcb988c31b, 6, Finished, Available, Finished)

# Fusion des Tables avec les Clés Primaires et Étrangères


In [5]:
# Jointure entre les commandes et les clients
df_orders_customers = df_orders_renamed.alias("commandes").join(
    df_customers_renamed.alias("clients"),
    df_orders_renamed["id_client_orders"] == df_customers_renamed["id_client"],
    "inner"
)

# Jointure entre les commandes et les articles de commande
df_orders_items_customers = df_orders_customers.alias("commandes_clients").join(
    df_order_items_renamed.alias("articles_commande"),
    df_orders_customers["id_commande_orders"] == df_order_items_renamed["id_commande_order_items"],
    "inner"
)

# Jointure avec les paiements
df_orders_items_customers_payments = df_orders_items_customers.alias("commandes_clients_articles").join(
    df_order_payments_renamed.alias("paiements"),
    df_orders_items_customers["id_commande_orders"] == df_order_payments_renamed["id_commande_order_payments"],
    "inner"
)

# Jointure avec les vendeurs
df_orders_items_customers_payments_sellers = df_orders_items_customers_payments.alias("commandes_clients_articles_paiements").join(
    df_sellers_renamed.alias("vendeurs"),
    df_orders_items_customers_payments["id_vendeur"] == df_sellers_renamed["id_vendeur"],
    "inner"
)

# Jointure avec les produits
df_orders_items_customers_payments_sellers_products = df_orders_items_customers_payments_sellers.alias("commandes_clients_articles_paiements_vendeurs").join(
    df_products_renamed.alias("produits"),
    df_orders_items_customers_payments_sellers["id_produit"] == df_products_renamed["id_produit"],
    "inner"
)

# Jointure avec la traduction des catégories
df_orders_items_customers_payments_sellers_products_category = df_orders_items_customers_payments_sellers_products.alias("commandes_clients_articles_paiements_vendeurs_produits").join(
    df_category_translation_renamed.alias("traductions_categories"),
    df_orders_items_customers_payments_sellers_products["categorie_produit"] == df_category_translation_renamed["categorie_produit"],
    "left"
)

# Jointure avec la géolocalisation
df_orders_items_customers_payments_sellers_products_category_geolocation = df_orders_items_customers_payments_sellers_products_category.alias("commandes_clients_articles_paiements_vendeurs_produits_categories").join(
    df_geolocation_renamed.alias("geolocalisation"),
    df_orders_items_customers_payments_sellers_products_category["code_postal_client"] == df_geolocation_renamed["code_postal_geolocalisation"],
    "left"
)

# Résultat final
df_final = df_orders_items_customers_payments_sellers_products_category_geolocation


StatementMeta(, e4d96399-a3a2-4659-953f-1afcb988c31b, 7, Finished, Available, Finished)

In [6]:
print(df_final.columns)


StatementMeta(, e4d96399-a3a2-4659-953f-1afcb988c31b, 8, Finished, Available, Finished)

['id_commande_orders', 'id_client_orders', 'statut_commande', 'date_achat_commande', 'date_approbation_commande', 'date_livraison_transporteur', 'date_livraison_client', 'date_livraison_estimee', 'id_client', 'id_unique_client', 'code_postal_client', 'ville_client', 'etat_client', 'id_commande_order_items', 'id_article_commande', 'id_produit', 'id_vendeur', 'date_limite_expedition', 'prix', 'valeur_fret', 'id_commande_order_payments', 'sequence_paiement', 'type_paiement', 'nombre_paiements', 'valeur_paiement', 'id_vendeur', 'code_postal_vendeur', 'ville_vendeur', 'etat_vendeur', 'id_produit', 'categorie_produit', 'longueur_nom_produit', 'longueur_description_produit', 'quantite_photos_produit', 'poids_produit_grammes', 'longueur_produit_cm', 'hauteur_produit_cm', 'largeur_produit_cm', 'categorie_produit', 'categorie_produit_anglais', 'code_postal_geolocalisation', 'latitude_geolocalisation', 'longitude_geolocalisation', 'ville_geolocalisation', 'etat_geolocalisation']


In [21]:
from pyspark.sql import functions as F

# Liste des colonnes numériques (ajustez la liste en fonction de vos données)
numerical_columns = [
    'prix', 'valeur_fret', 'valeur_paiement', 'longueur_nom_produit',
    'longueur_description_produit', 'quantite_photos_produit', 'poids_produit_grammes',
    'longueur_produit_cm', 'hauteur_produit_cm', 'largeur_produit_cm',
    'latitude_geolocalisation', 'longitude_geolocalisation'
]

# Sélectionner uniquement les colonnes numériques
df_numeric = df_final.select(numerical_columns)

# Collecter les données dans un DataFrame Pandas
df_pandas = df_numeric.toPandas()

# Calculer la matrice de corrélation
correlation_matrix = df_pandas.corr()

# Afficher la matrice de corrélation
print(correlation_matrix)


StatementMeta(, e4d96399-a3a2-4659-953f-1afcb988c31b, 23, Finished, Available, Finished)

                                  prix  valeur_fret  valeur_paiement  \
prix                          1.000000     0.427033         0.749702   
valeur_fret                   0.427033     1.000000         0.375656   
valeur_paiement               0.749702     0.375656         1.000000   
longueur_nom_produit          0.015927     0.023894         0.002594   
longueur_description_produit  0.203567     0.096027         0.156875   
quantite_photos_produit       0.054607     0.025393         0.003306   
poids_produit_grammes         0.342779     0.642492         0.303701   
longueur_produit_cm           0.146236     0.329643         0.137449   
hauteur_produit_cm            0.220690     0.410154         0.212541   
largeur_produit_cm            0.173085     0.341786         0.149490   
latitude_geolocalisation      0.038108     0.182289         0.045080   
longitude_geolocalisation     0.014737     0.057945         0.012428   

                              longueur_nom_produit  \
prix     

In [26]:
# Sélectionner les colonnes fortement corrélées
strong_correlated_columns = [
    'longitude_geolocalisation', 'valeur_fret', 'longueur_description_produit',
    'poids_produit_grammes', 'prix', 'hauteur_produit_cm', 'largeur_produit_cm',
    'longueur_produit_cm', 'longueur_nom_produit', 'valeur_paiement',
    'quantite_photos_produit', 'latitude_geolocalisation'
]

# Créer un sous-ensemble du DataFrame avec ces colonnes
df_strong_corr = df_pandas[strong_correlated_columns]

# Calculer la matrice de corrélation
correlation_matrix = df_strong_corr.corr()

# Afficher la matrice de corrélation
print(correlation_matrix)




StatementMeta(, e4d96399-a3a2-4659-953f-1afcb988c31b, 28, Finished, Available, Finished)

                              longitude_geolocalisation  valeur_fret  \
longitude_geolocalisation                      1.000000     0.057945   
valeur_fret                                    0.057945     1.000000   
longueur_description_produit                  -0.008212     0.096027   
poids_produit_grammes                          0.004281     0.642492   
prix                                           0.014737     0.427033   
hauteur_produit_cm                            -0.005806     0.410154   
largeur_produit_cm                            -0.005141     0.341786   
longueur_produit_cm                           -0.008027     0.329643   
longueur_nom_produit                          -0.007676     0.023894   
valeur_paiement                                0.012428     0.375656   
quantite_photos_produit                       -0.000318     0.025393   
latitude_geolocalisation                       0.421017     0.182289   

                              longueur_description_produit  \
l

### Architecture du Cluster
### Master Node : Contient df_final pour l'analyse centralisée.
### Slave Nodes : Chaque nœud stocke une table source :
### Slave 1: df_customers_renamed
### Slave 2: df_sellers_renamed
### Slave 3: df_products_renamed

In [19]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("Cluster_PySpark") \
    .master("local[*]") \
    .config("spark.executor.memory", "2g") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()



StatementMeta(, e4d96399-a3a2-4659-953f-1afcb988c31b, 21, Finished, Available, Finished)

In [20]:
df_orders_items_customers_payments_sellers = df_orders_items_customers_payments.join(
    df_sellers.alias("sellers"),
    df_orders_items_customers_payments["seller_id"] == df_sellers["seller_id"], 
    "inner"
)

StatementMeta(, ee478efe-534c-4094-bd55-71d8fae64427, 23, Finished, Available, Finished)

In [21]:
df_orders_items_customers_payments_sellers_products = df_orders_items_customers_payments_sellers.join(
    df_products.alias("products"),
    df_orders_items_customers_payments_sellers["product_id"] == df_products["product_id"], 
    "inner"
)

StatementMeta(, ee478efe-534c-4094-bd55-71d8fae64427, 24, Finished, Available, Finished)

In [31]:
df_orders_items_customers_payments_sellers_products_category = df_orders_items_customers_payments_sellers_products.join(
    df_category_translation.alias("category_translation"),
    df_orders_items_customers_payments_sellers_products["product_category_name"] == df_category_translation["product_category_name"], 
    "left"
)

StatementMeta(, ee478efe-534c-4094-bd55-71d8fae64427, 34, Finished, Available, Finished)

## Affichage des donnes apres jointure


In [33]:
df_orders_customers.show(5)


StatementMeta(, ee478efe-534c-4094-bd55-71d8fae64427, 36, Finished, Available, Finished)

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+--------------------+--------------------+------------------------+--------------------+--------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+--------------------+--------------------+------------------------+--------------------+--------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-

In [34]:
df_orders_items_customers.show(2)

StatementMeta(, ee478efe-534c-4094-bd55-71d8fae64427, 37, Finished, Available, Finished)

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+--------------------+--------------------+------------------------+-------------+--------------+--------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|         customer_id|  customer_unique_id|customer_zip_code_prefix|customer_city|customer_state|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date|price|freight_value|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+--

# Configuration SparkSession pour me connecter au cluster Docker

In [17]:
from pyspark.sql import SparkSession


spark = SparkSession.builder \
    .appName("PowerBI_Spark_Integration") \
    .master("spark://localhost:7077") \
    .config("spark.executor.memory", "1g") \
    .config("spark.executor.cores", "2") \
    .getOrCreate()


StatementMeta(, e4d96399-a3a2-4659-953f-1afcb988c31b, 19, Finished, Available, Finished)

# stocker la grande base de donne sur mon worker 1

In [18]:

# Définir le chemin de sortie HDFS (sur le worker 1)
output_path = "hdfs://172.29.0.7:9000/spark_data/df_customers_renamed.parquet"

# Enregistrez le DataFrame sur HDFS
df_customers_renamed.write.parquet(output_path)

print("Le DataFrame 'df_customers_renamed' a été enregistré avec succès sur le worker.")


StatementMeta(, e4d96399-a3a2-4659-953f-1afcb988c31b, 20, Finished, Cancelled, Cancelled)

KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages/py4j/java_gateway.py", line 1217, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/home/trusted-service-user/cluster-env/trident_env/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt
